In [ ]:

from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np 
import torch
import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
shutil.move("/content/gdrive/MyDrive/final_data.csv", "/content")

'/content/final_data.csv'

In [ ]:

!pip install pytorch
!pip install -U jina==2.2.5
!pip install torch==1.9.0
!pip install transformers==4.5.1
!pip install sentence_transformers
#!pip install https://download.pytorch.org/whl/torch_stable.html



  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-iuvs_8gq/pytorch_8c942ec2c4c54de68903f93e713dae05/setup.py'"'"'; __file__='"'"'/tmp/pip-install-iuvs_8gq/pytorch_8c942ec2c4c54de68903f93e713dae05/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-q98nbsl0/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pytorch Check the logs for full command output.
     |█████████████████████

In [ ]:
from sentence_transformers import SentenceTransformer
model= SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v1')

In [ ]:
df = pd.read_csv('final_data.csv')
df.columns=['Text','Speaker','Episode','Season','Show']
df.head()

,Text,Speaker,Episode,Season,Show
0,Originally written by Marta Kauffman and David...,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
1,Transcribed by guineapig.,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
3,There's nothing to tell! He's just some guy I...,MONICA,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
4,"C'mon, you're going out with the guy! There's...",JOEY,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends


In [ ]:
df.dropna(subset=['Text'],inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
df

,Text,Speaker,Episode,Season,Show
0,Originally written by Marta Kauffman and David...,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
1,Transcribed by guineapig.,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
3,There's nothing to tell! He's just some guy I...,MONICA,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
4,"C'mon, you're going out with the guy! There's...",JOEY,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
...,...,...,...,...,...
289168,Morning. Hi.,NaN,Episode-01-Nangnangnangnang,Season-11,Two-And-Half-Men
289169,You girls have a good night?,NaN,Episode-01-Nangnangnangnang,Season-11,Two-And-Half-Men
289170,"Oh, yeah.",NaN,Episode-01-Nangnangnangnang,Season-11,Two-And-Half-Men
289171,I'm surprised I can even talk.,NaN,Episode-01-Nangnangnangnang,Season-11,Two-And-Half-Men


# New section

In [ ]:
docs = DocumentArray()
for ind in range(100):
    text = df.iloc[ind, 0]
    speaker = df.iloc[ind, 1]
    episode= df.iloc[ind, 2]
    season= df.iloc[ind, 3]
    show= df.iloc[ind, 4]
    doc = Document(text = text)
    doc.tags['speaker'] = speaker
    doc.tags['episode'] = episode
    doc.tags['season'] = season
    doc.tags['show'] = show
    docs.append(doc)
    
docs

<jina.types.arrays.document.DocumentArray (length=100) at 140514013245776>

In [ ]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name="text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
        
    )
    .add(
        name="text_indexer",
        uses='jinahub://SimpleIndexer',
        #uses_with={'device': 'cuda'},
        
    )
)

In [ ]:
docs=docs[:100]

In [ ]:

docs[0]

In [ ]:
print(docs[4].json())

{
  "id": "9de6e7a0-51bd-11ec-a72d-0242ac1c0002",
  "mime_type": "text/plain",
  "tags": {
    "episode": "Episode-01-The One Where Monica Gets a New Roommate (The Pilot)",
    "season": "Season-01",
    "show": "Friends",
    "speaker": "JOEY"
  },
  "text": " C'mon, you're going out with the guy! There's gotta be something wrong with him!"
}


In [ ]:
len(docs)

100

In [ ]:
!rm -rf workspace

with flow:
    flow.index(
      inputs=docs,
        
        #parameters = {'name' : 'somethign', 'xyz' : 'fsdfsdfsa'},
        gpus='all'
  )

Output()

⠴ Unpacking TransformerTorchEncoder ...

Output()

⠏ Downloading SimpleIndexer ...

           Flow@384[I]:🎉 Flow is ready to use!
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:41093
	🔒 Private network:	172.28.0.2:41093
	🌐 Public address:	35.226.65.138:41093
   text_encoder@655[E]:TypeError('`traversal_paths` needs to be `Sequence[str]`')
 add "--quiet-error" to suppress the exception details
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/zmq/zed.py", line 285, in _msg_callback
    processed_msg = self._callback(msg)
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/zmq/zed.py", line 271, in _callback
    msg = self._post_hook(self._handle(self._pre_hook(msg)))
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/zmq/zed.py", line 226, in _handle
    peapod_name=self.name,
  File "/usr/local/lib/python3.7/dist-packages/jina/peapods/runtimes/request_handlers/data_request_handler.py", line 165, in handle
    field='groundtruths',
  File "/usr/local/lib/python3.7/dist-packages/jina

In [ ]:

def query_text():
    query = Document(text = input("Enter the Text - "))
    print()
    with flow:
        response = flow.search(inputs = query,parameters={'limit':20},uses_with ={'device':'cpu'} ,return_results = True)
    matches = response[0].docs[0].matches

    for i in matches[:5]:
        print()
        print(str(i.text),"\n")
        print("series :", i.tags['show'],"\n")
        print("season :", i.tags['season'],"\n")
        print("episode :", i.tags['episode'],"\n")
        print("speaker :", i.tags['speaker'],"\n")
    
        print("_________________________________________________________________________________________________________________-\n")


In [ ]:
query_text()

Enter the Text - hello

           Flow@537[I]:🎉 Flow is ready to use!
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:44253
	🔒 Private network:	172.28.0.2:44253
	🌐 Public address:	35.245.233.26:44253

Charlie. 

series : Two-And-Half-Men 

season : Season-06 

episode : Episode-02-Pie Hole, Herb 

speaker : nan 

_________________________________________________________________________________________________________________-


- Yup. 

series : Two-And-Half-Men 

season : Season-06 

episode : Episode-02-Pie Hole, Herb 

speaker : nan 

_________________________________________________________________________________________________________________-


Hey, Charlie. 

series : Two-And-Half-Men 

season : Season-06 

episode : Episode-02-Pie Hole, Herb 

speaker : nan 

_________________________________________________________________________________________________________________-


Thanks, Mom. 

series : Two-And-Half-Men 

season : Season-06 

episode : Episode-02-Pie Hole, Herb 

spe

___________________

In [ ]:
!